In [3]:
from pathlib import Path

import dask
from dask.graph_manipulation import bind
from dask.distributed import Client

from convert_composer_to_hf import ConvertComposerToHfConfig, convert_composer_to_hf
from evaluate_model import EvaluationConfig, evaluate_model
from pipeline import PipelineConfig

# from continued_pretraining import run_continued_pretraining
from further.experiments.scripts.dispatch import launch_workflow

configs = {
    "7b_to_370m": PipelineConfig(
        from_model_name="NousResearch/Llama-2-7b-hf",
        from_model_size="7b",
        to_model_size="370m",
    ),
    "1.3b_to_370m": PipelineConfig(
        from_model_name="princeton-nlp/Sheared-LLaMA-1.3B",
        from_model_size="1.3b",
        to_model_size="370m",
    ),
}


@dask.delayed
def export_task(group: str, config: ConvertComposerToHfConfig):
    launch_workflow(convert_composer_to_hf, group=group, gpu=2, no_follow=True)(config)


@dask.delayed
def eval_task(group: str, config: EvaluationConfig):
    launch_workflow(evaluate_model, group=group, gpu=2, no_follow=True)(config)


if __name__ == "__main__":
    client = Client()
    print(client.dashboard_link)

    futures = []

    for exp_name, config in configs.items():
        assert config.to_model_dims is not None

        """Create configs"""

        # original
        original_eval_config = config.from_model_eval_config

        # pruned
        pruned_export_config = config.pruned_to_hf_config
        pruned_eval_config = config.pruning_eval_config

        # continued pretraining
        checkpoint_paths = config.continued_pretraining_config.get_checkpoint_paths()
        export_and_eval_configs = []
        for checkpoint_path in checkpoint_paths:
            export_config = ConvertComposerToHfConfig(
                model_path=str(checkpoint_path),
                model_name=checkpoint_path.name,
                network_dims=config.to_model_dims,
            )
            eval_config = EvaluationConfig(model=export_config.output_path)
            export_and_eval_configs.append((export_config, eval_config))

        """Run tasks"""

        group = f"{exp_name}-original"
        futures.append(eval_task(group, original_eval_config))

        group = f"{exp_name}-pruned"
        future = export_task(group, pruned_export_config)
        futures.append(bind(eval_task, [future])(group, pruned_eval_config))

        for export_config, eval_config in export_and_eval_configs:
            ba = config.continued_pretraining_config.get_checkpoint_id(
                export_config.model_path
            )
            group = f"{exp_name}-pretraining-ba{ba}"
            future = export_task(group, export_config)
            futures.append(bind(eval_task, [future])(group, eval_config))

    dask.compute(*futures)

http://127.0.0.1:8787/status
Launching workflow: convert_composer_to_hfLaunching workflow: evaluate_model
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf

Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: evaluate_model
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_hf
Launching workflow: convert_composer_to_h

/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is depr

Preparing files to send to master... 1.1MB and 84 fileses                                                     sreparing files to send to master... 7.0KB and 14 files reparing files to send to master... 1.1MB and 80 filesPreparing files to send to master... 1.1MB and 66 files                                                       
Preparing files to send to master... 1.1MB and 84 filesPreparing files to send to master... 1.1MB and 84 filesles
                                                       
                                                       es                                                                                                              
                                                       
Preparing files to send to master... 1.1MB and 83 files
Preparing files to send to master... 45.8KB and 27 filesreparing files to send to master... 1.1MB and 84 files
                                                                                                            

/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/trial.py:579: FutureWarning: 'TrialOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/experiment.py:78: FutureWarning: 'ExperimentOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/model.py:186: FutureWarning: 'ModelOrderBy' is deprecated and will be removed in a future release. Please

Preparing files to send to master... 1.1MB and 84 files  
Experiment started at http://mlds-determined.us.rdlabs.hpecorp.net:8080/det/experiments/13505/logs
Experiment 13481 completed with state ExperimentState.COMPLETED
Launching workflow: convert_composer_to_hf


...

Experiment 13539 completed with state ExperimentState.COMPLETED
Experiment 13487 completed with state ExperimentState.COMPLETED
Launching workflow: evaluate_model


/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/trial.py:579: FutureWarning: 'TrialOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/experiment.py:78: FutureWarning: 'ExperimentOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/model.py:186: FutureWarning: 'ModelOrderBy' is deprecated and will be removed in a future release. Please

Preparing files to send to master... 1.1MB and 84 files  
Experiment started at http://mlds-determined.us.rdlabs.hpecorp.net:8080/det/experiments/13543/logs
Experiment 13540 completed with state ExperimentState.COMPLETED
Experiment 13541 completed with state ExperimentState.COMPLETED
Experiment 13512 completed with state ExperimentState.COMPLETED
Launching workflow: evaluate_model


/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/checkpoint/_checkpoint.py:62: FutureWarning: 'CheckpointOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/trial.py:579: FutureWarning: 'TrialOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/experiment.py:78: FutureWarning: 'ExperimentOrderBy' is deprecated and will be removed in a future release. Please use 'experimental.OrderBy' instead.
  warnings.warn(
/nvmefs1/daranhe/.conda/envs/default/lib/python3.10/site-packages/determined/common/experimental/model.py:186: FutureWarning: 'ModelOrderBy' is deprecated and will be removed in a future release. Please

Preparing files to send to master... 1.1MB and 84 files  
Experiment started at http://mlds-determined.us.rdlabs.hpecorp.net:8080/det/experiments/13544/logs
Experiment 13542 completed with state ExperimentState.COMPLETED
Experiment 13543 completed with state ExperimentState.COMPLETED
Experiment 13544 completed with state ExperimentState.COMPLETED
